# 最终完整代码

In [ ]:
from selenium import webdriver
import time
import requests,re

# 建立数据库
from pony.orm import *
db = Database()
class Star(db.Entity):
    id = PrimaryKey(int, column='id', auto=True)
    name = Optional(str, column='name')
    gender = Optional(str, column='gender')
    href = Optional(str, column='href')
    year = Optional(int, column='year')
    month = Optional(int, column='month')
    day = Optional(int, column='day')
    xz = Optional(str, column='xz')
    html = Optional(str, column='html')
    address = Optional(str, column='address')
    height = Optional(float, column='height')
# db.generate_mapping()

db.bind(provider = 'sqlite', filename = 'e:/python/star.sqlite')
db.generate_mapping(create_tables = True)
set_sql_debug(True)

# 打开明星网页
driver = webdriver.Chrome()
url = 'https://www.baidu.com/s?wd=%E6%98%8E%E6%98%9F&rsv_spt=1&rsv_iqid=0xbe4b76860031fb66&issp=1&f=8&rsv_bp=1&rsv_idx=2&ie=utf-8&rqlang=&tn=baiduhome_pg&ch=&rsv_enter=1&rsv_dl=ib&inputT=2978'
driver.get(url)

# 点击女，内地
driver.find_element_by_xpath("//div[@id='1']/div/div/div/div[2]/p/span[4]").click()
driver.find_element_by_xpath("//div[@id='1']/div/div/div/div[2]/p[2]/span[3]").click()

time.sleep(1)

# 采集明星的href、neme、html并入库
@db_session
def task1():
    reobj = re.compile(r'<p class="c-gap-top-small"><a href="([\d\D]*?)" title="(.{1,20})" target="_blank">[\d\D]*?</a></p>')
    for match in reobj.finditer(html):
        get_href = match.group(1)
        get_name = match.group(2)
        head = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}
        url = 'https://baike.baidu.com/item/' + get_name
        r = requests.get(url,headers = head)
        r.encoding = 'utf-8'
        get_html = r.text
        # 如果获取的明星百科html中没有如下字，则可能找到了多个，于是在该网页中找到明星后面的数字添加到url中再获取明星百科html
        if "星&nbsp;&nbsp;&nbsp;&nbsp;座" not in get_html and "身&nbsp;&nbsp;&nbsp;&nbsp;高" not in get_html and "出生地" not in get_html and "出生日期" not in get_html and "基本信息" not in get_html and "出生年月" not in get_html:
            reobj = re.compile(r'<a target=_blank href="[\d\D]*?([0-9]*?)" data-lemmaid="[\d\D]*?">[\d\D]*?</a>')
            for match in reobj.finditer(get_html):
                url = 'https://baike.baidu.com/item/' + get_name + '/' + match.group(1)
                r = requests.get(url,headers = head)
                r.encoding = 'utf-8'
                get_html = r.text
                break
        s = Star(name = get_name,gender = "女",href = "https://www.baidu.com" + get_href,html = get_html)
        db.commit()

#判断下一页按钮是否存在，若存在则点击下一页
while True:
    html = driver.page_source# 获取网页源代码
    task1()
    x = ''
    reobj2 = re.compile(r'<span class="opui-page-next OP_LOG_BTN" style="([\d\D]*?)">下一页</span>')
    for match2 in reobj2.finditer(html):
        x = match2.group(1)
    if len(x) > 0:
        driver.quit()
        break
    else:
        driver.find_element_by_xpath("//div[@id='1']/div/div/div[2]/div[2]/p/span[6]").click()#点击下一页
        time.sleep(1)

# 采集明星的星座并入库
@db_session
def get_xz(html):
    sxz = ""
    reobj = re.compile(r"""星&nbsp;&nbsp;&nbsp;&nbsp;座</dt>
<dd class="basicInfo-item value">
([\d\D]*?)
</dd>""")
    for match in reobj.finditer(html):
        sxz = match.group(1)
        #如果获取到的星座中有超链接，则做如下处理
        if '<a target' in sxz:
            reobj = re.compile(r"""星&nbsp;&nbsp;&nbsp;&nbsp;座</dt>
<dd class="basicInfo-item value">
<a target=["]?_blank["]? href="[\d\D]*?">([\d\D]*?)</a>
</dd>""")
            for match in reobj.finditer(html):
                sxz = match.group(1)
        elif '<sup class=' in sxz:
            sxz = sxz[:3]
        s.xz = sxz.replace("&nbsp;","")#去掉获取到的星座中的"&nbsp;"
        db.commit()
        
# 采集明星的身高并入库
@db_session
def get_height(html):
    sheight = 0
    reobj = re.compile(r"""身&nbsp;&nbsp;&nbsp;&nbsp;高</dt>
<dd class="basicInfo-item value">
[&nbsp;]*?([\d.]*?)[cmCM厘米<sup ]{1,2}[\d\D]*?
</dd>""", re.DOTALL)
    for match in reobj.finditer(html):
        if len(match.group(1)) > 5:
            reobj = re.compile(r"""身&nbsp;&nbsp;&nbsp;&nbsp;高</dt>
<dd class="basicInfo-item value">
([\d\D]*?)<a target=["]?_blank["]? href=""")
            for match in reobj.finditer(html):
                sheight = match.group(1)
        elif match.group(1) == "":
            reobj = re.compile(r"""身&nbsp;&nbsp;&nbsp;&nbsp;高</dt>
<dd class="basicInfo-item value">
[&nbsp;]*?([\d.]*?)[cmCM厘米<sup ]{1,2}
</dd>""", re.DOTALL)
            for match in reobj.finditer(html):
                if str(match.group(1)).isdigit() == True:
                    sheight = match.group(1)
        else:
            sheight = match.group(1)
    if sheight != 0 :
        s.height = str(sheight).replace("&nbsp;","")#去掉获取到的身高中的"&nbsp;"
        db.commit()
        
# 采集明星的出生地并入库
@db_session
def get_address(html):
    saddress = ""
    reobj = re.compile(r"""出生地</dt>
<dd class="basicInfo-item value">
([\d\D]*?)
</dd>""")
    for match in reobj.finditer(html):
        if match.group(1).count('<a target=') == 1:
            reobj = re.compile(r"""出生地</dt>
<dd class="basicInfo-item value">
([\d\D]*?)<a target=["]?_blank["]? href="[\d\D]*?">([\d\D]*?)</a>
</dd>""")
            for match in reobj.finditer(html):
                if len(match.group(1)) + len(match.group(2)) < 20:
                    saddress = match.group(1) + match.group(2)
                    
            reobj = re.compile(r"""出生地</dt>
<dd class="basicInfo-item value">
<a target=["]?_blank["]? href="[\d\D]*?">([\d\D]*?)</a>([\d\D]*?)
</dd>""")
            for match in reobj.finditer(html):
                if len(match.group(1)) + len(match.group(2)) < 20:
                    saddress = match.group(1) + match.group(2)
                    
                    
        elif match.group(1).count('<a target=') == 2:
            reobj = re.compile(r"""出生地</dt>
<dd class="basicInfo-item value">
<a target=["]?_blank["]? href="[\d\D]*?">([\d\D]*?)</a>[&nbsp;]*?([\d\D]*?)<a target=["]?_blank["]? href="[\d\D]*?">([\d\D]*?)</a>([\d\D]*?)
</dd>""")
            for match in reobj.finditer(html):
                if len(match.group(1)) + len(match.group(2))+ len(match.group(3)) + len(match.group(4)) < 20:
                    saddress = match.group(1) + match.group(2) + match.group(3) + match.group(4)
                
                
        elif match.group(1).count('<a target=') == 3:
            reobj = re.compile(r"""<a target=["]?_blank["]? href="/item/%E4%BA%91%E5%8D%97">([\d\D]*?)</a><a target=["]?_blank["]? href="[\d\D]*?">([\d\D]*?)</a><a target=["]?_blank["]? href="/item/%E6%A5%9A%E9%9B%84%E5%B8%82">([\d\D]*?)</a>
</dd>""")
            for match in reobj.finditer(html):
                if len(match.group(1)) + len(match.group(2)) +len(match.group(3)) < 20:
                    saddress = match.group(1) + match.group(2) + match.group(3)
                
                
        elif """<sup class="sup--normal" data-sup="1" data-ctrmap=":1,">""" in match.group(1):
            reobj = re.compile(r"""出生地</dt>
<dd class="basicInfo-item value">
([\d\D]*?)<sup class="sup--normal" data-sup="1" data-ctrmap=":1,">
\[1\]</sup><a class="sup-anchor" name="ref_\[1\]_10531">&nbsp;</a>
</dd>""")
            for match in reobj.finditer(html):
                if len(match.group(1))< 20:
                    saddress = match.group(1)
                
                
        elif """<dt class="basicInfo-item name">出生地</dt>""" in match.group(1):
            reobj = re.compile(r"""<dt class="basicInfo-item name">出生地</dt>
<dd class="basicInfo-item value">
<a target="[\d\D]*?">([\d\D]*?)</a>([\d\D]*?)<a target=["]?_blank["]? href="[\d\D]*?">([\d\D]*?)</a>([\d\D]*?)
</dd>""")
            for match in reobj.finditer(html):
                if len(match.group(1)) + len(match.group(2)) + len(match.group(3)) + len(match.group(4)) < 20:
                    saddress = match.group(1) + match.group(2) + match.group(3) + match.group(4)
                
                
        elif """<sup class=""" in match.group(1):
            reobj = re.compile(r"""出生地</dt>
<dd class="basicInfo-item value">
([\d\D]*?)<sup class="[\d\D]*?</a>
</dd>""")
            for match in reobj.finditer(html):
                if len(match.group(1)) < 20:
                    saddress = match.group(1)

                
        else:
            saddress = match.group(1)
    s.address = saddress.replace("&nbsp;","")
    db.commit()
    
# 采集明星的出生年并入库
@db_session
def get_year(html):
    syear = 0
    x = ""
    reobj = re.compile(r"""出生日期</dt>
<dd class="basicInfo-item value[\d\D]*?">
([\d\D]*?)[年:\-. /][\d\D]*?
</dd>""", re.DOTALL)
    for match in reobj.finditer(html):
        if match.group(1) is None:
            break
        elif "月" in match.group(1) or "日" in match.group(1):
            syear = 0
        elif len(match.group(1)) == 4:
            syear = match.group(1)
        elif "<a target=" in match.group(1):
            if str(match.group(1)[-4:]).isdigit() == True:
                syear = match.group(1)[-4:]
            else:
                reobj = re.compile(r"""出生日期</dt>
<dd class="basicInfo-item value">
<a target=[\d\D]*?">([\d\D]*?)</a>[年.\- /][\d\D]*?
</dd>""", re.DOTALL)
                for match in reobj.finditer(html):
                    if str(match.group(1)).isdigit() == True:
                        syear = match.group(1)
                    
    reobj = re.compile(r"""出生日期</dt>
<dd class="basicInfo-item value">
生于([\d\D]*?)[年:\-. /][\d\D]*?
</dd>""", re.DOTALL)
    for match in reobj.finditer(html):
        for i in match.group(1):
            if i in sz:
                x += str(sz.find(i)+1)
        syear = x
    if str(syear).isdigit() == True and syear != 0:
        s.year = str(syear).replace("&nbsp;","")#去掉获取到的出生年中的"&nbsp;"
        db.commit()

# 采集明星的出生月并入库
@db_session
def get_month(html):
    smonth = 0
    reobj = re.compile(r"""出生日期</dt>
<dd class="basicInfo-item value">
[\d\D]*?[年:\-. /]?([\d\D]*?)[月\-./][\d\D]*?
</dd>""", re.DOTALL)
    for match in reobj.finditer(html):
        smonth = match.group(1)
        if '年' in match.group(1) and len(re.split('年',match.group(1))[1].strip()) < 6:
            if re.split('年',match.group(1))[1].strip().isdigit() == True:
                smonth = re.split('年',match.group(1))[1].strip()
            elif re.split('年',match.group(1))[1].strip() in sz:
                smonth = str(sz.find(re.split('年',match.group(1))[1])+1)
            elif '</a>' in match.group(1) or ""and len(match.group(1)) < 60:
                smonth = str(re.split('年',match.group(1))[1]).strip().strip('</a>')
            elif match.group(1).count("年") == 2:
                smonth = str(re.split('年',match.group(1))[2]).strip()[0:]
                if len(smonth) >2:
                    smonth = smonth[-2:]
            else:
                smonth = 0
        elif "历" in match.group(1) and len(str(re.split('历',match.group(1))[1]).strip()) < 3:
            if re.split('历',match.group(1))[1].strip().isdigit() == True:
                smonth = re.split('历',match.group(1))[1].strip()
            elif re.split('历',match.group(1))[1].strip() in sz:
                smonth = sz.find(str(re.split('历',match.group(1))[1]).strip())+1
        elif '<a target=' in smonth and len(str(smonth)) < 55:
            reobj = re.compile(r"""出生日期</dt>
<dd class="basicInfo-item value">
<a target=[\d\D]*?年([\d\D]*?)月[\d\D]*?""", re.DOTALL)
            for match in reobj.finditer(html):
                smonth = match.group(1)
        elif '<a target=' in smonth and len(str(smonth)) > 55:
            reobj = re.compile(r"""出生日期</dt>
<dd class="basicInfo-item value">
([\d\D]*?)月[\d\D]*?
</dd>""", re.DOTALL)
            for match in reobj.finditer(html):
                if str(match.group(1)[-2:]).isdigit() == True:
                    smonth = match.group(1)[-2:]
                else:
                    smonth = match.group(1)[-1:]
        elif str(smonth).isdigit == True and len(str(smonth)) == 4:
            reobj = re.compile(r"""出生日期</dt>
<dd class="basicInfo-item value">
[\d\D]*?[年:\-. /]([\d\D]*?)[月\-. /][\d\D]*?
</dd>""", re.DOTALL)
            for match in reobj.finditer(html):
                if str(match.group(1)).sidigit == True:
                    smonth = match.group(1)

        if len(match.group(1)) == 5 and '</a>' in match.group(1)and len(re.findall('[0-9]',match.group(1))[0])< 3:
            smonth = re.findall('[0-9]',match.group(1))[0]
        if str(smonth).isdigit() != True:
            smonth = 0
    if smonth != 0:
        s.month = str(smonth).replace("&nbsp;","")#去掉获取到的出生月中的"&nbsp;"
        db.commit()

# 采集明星的出生日并入库
@db_session
def get_day(html):
    sday = 0
    x = ""
    reobj = re.compile(r"""出生日期</dt>
<dd class="basicInfo-item value">
[\d\D]*?[年:\-. /]?[\d\D]*?[月:\-. /]([\d\D]*?)[日.-号][&nbsp;]?[\d\D]*?
</dd>""", re.DOTALL)
    for match in reobj.finditer(html): 
        if match.group(1) is None or len(match.group(1))>2 or match.group(1) == "":
            sday = 0
        else:
            sday = match.group(1)
    
    reobj = re.compile(r"""出生日期</dt>
<dd class="basicInfo-item value">
[\d\D]*?[年-]?[\d\D]*?[月.-]([\d\D]{2}?)[日.-号][&nbsp;]?[\d\D]*?
</dd>""", re.DOTALL)
    for match in reobj.finditer(html):
        for i in match.group(1):
            if i.isdigit() == True:
                x += i
        if len(x) == 2:
            sday = match.group(1)
        else:
            reobj = re.compile(r"""出生日期</dt>
<dd class="basicInfo-item value">
[\d\D]*?[年-]?[\d\D]*?[月.-]([\d\D]{1}?)[日.-号][&nbsp;]?[\d\D]*?
</dd>""", re.DOTALL)
            for match in reobj.finditer(html):
                if match.group(1).strip().isdigit() == True:
                    sday = match.group(1).strip()
                else:
                    reobj = re.compile(r"""出生日期</dt>
<dd class="basicInfo-item value">
[\d\D]*?月([\d\D]*?)[日号]
</dd>""", re.DOTALL)
                    for match in reobj.finditer(html):
                        if match.group(1).isdigit() == True:
                            sday = match.group(1)
                        else:
                            reobj = re.compile(r"""出生日期</dt>
<dd class="basicInfo-item value">
[\d\D]*?[月:\-. /][\d\D]*?[月:\-. /]([\d\D]*?)
</dd>""", re.DOTALL)
                            for match in reobj.finditer(html):
                                if str(match.group(1)).isdigit == True:
                                    sday = match.group(1)
    if str(sday).isdigit() != True:
        sday = 0
    if sday != 0:
        s.day = str(sday).replace("&nbsp;","")#去掉获取到的出生日中的"&nbsp;"
        db.commit()

# 运行
sz = "一二三四五六七八九"
with db_session:
    for s in Star.select():
        get_height(s.html)
        get_xz(s.html)
        get_address(s.html)
        get_year(s.html)
        get_month(s.html)
        get_day(s.html)

# 建立数据库

In [1]:
from pony.orm import *
db = Database()
class Star(db.Entity):
    id = PrimaryKey(int, column='id', auto=True)
    name = Optional(str, column='name')
    gender = Optional(str, column='gender')
    href = Optional(str, column='href')
    year = Optional(int, column='year')
    month = Optional(int, column='month')
    day = Optional(int, column='day')
    xz = Optional(str, column='xz')
    html = Optional(str, column='html')
    address = Optional(str, column='address')
    height = Optional(float, column='height')
# db.generate_mapping()

db.bind(provider = 'sqlite', filename = 'e:/python/star.sqlite')
db.generate_mapping(create_tables = True)
set_sql_debug(True)

# 打开明星网页

In [ ]:
driver = webdriver.Chrome()
url = 'https://www.baidu.com/s?wd=%E6%98%8E%E6%98%9F&rsv_spt=1&rsv_iqid=0xbe4b76860031fb66&issp=1&f=8&rsv_bp=1&rsv_idx=2&ie=utf-8&rqlang=&tn=baiduhome_pg&ch=&rsv_enter=1&rsv_dl=ib&inputT=2978'
driver.get(url)

# 点击女，内地

In [ ]:
driver.find_element_by_xpath("//div[@id='1']/div/div/div/div[2]/p/span[4]").click()
driver.find_element_by_xpath("//div[@id='1']/div/div/div/div[2]/p[2]/span[3]").click()

# 采集明星的href、neme、html并入库

In [ ]:
@db_session
def task1():
    reobj = re.compile(r'<p class="c-gap-top-small"><a href="([\d\D]*?)" title="(.{1,20})" target="_blank">[\d\D]*?</a></p>')
    for match in reobj.finditer(html):
        get_href = match.group(1)
        get_name = match.group(2)
        head = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}
        url = 'https://baike.baidu.com/item/' + get_name
        r = requests.get(url,headers = head)
        r.encoding = 'utf-8'
        get_html = r.text
        # 如果获取的明星百科html中没有如下字，则可能找到了多个，于是在该网页中找到明星后面的数字添加到url中再获取明星百科html
        if "星&nbsp;&nbsp;&nbsp;&nbsp;座" not in get_html and "身&nbsp;&nbsp;&nbsp;&nbsp;高" not in get_html and "出生地" not in get_html and "出生日期" not in get_html and "基本信息" not in get_html and "出生年月" not in get_html:
            reobj = re.compile(r'<a target=_blank href="[\d\D]*?([0-9]*?)" data-lemmaid="[\d\D]*?">[\d\D]*?</a>')
            for match in reobj.finditer(get_html):
                url = 'https://baike.baidu.com/item/' + get_name + '/' + match.group(1)
                r = requests.get(url,headers = head)
                r.encoding = 'utf-8'
                get_html = r.text
                break
        s = Star(name = get_name,gender = "女",href = "https://www.baidu.com" + get_href,html = get_html)
        db.commit()

# 判断下一页按钮是否存在，若存在则点击下一页

In [ ]:
while True:
    html = driver.page_source# 获取网页源代码
    task1()
    x = ''
    reobj2 = re.compile(r'<span class="opui-page-next OP_LOG_BTN" style="([\d\D]*?)">下一页</span>')
    for match2 in reobj2.finditer(html):
        x = match2.group(1)
    if len(x) > 0:#则为最后一页
        driver.quit()
        break
    else:
        driver.find_element_by_xpath("//div[@id='1']/div/div/div[2]/div[2]/p/span[6]").click()#点击下一页
        time.sleep(1)

# 采集明星的星座并入库

In [ ]:
@db_session
def get_xz(html):
    sxz = ""
    reobj = re.compile(r"""星&nbsp;&nbsp;&nbsp;&nbsp;座</dt>
<dd class="basicInfo-item value">
([\d\D]*?)
</dd>""")
    for match in reobj.finditer(html):
        sxz = match.group(1)
        if '<a target' in sxz:
            reobj = re.compile(r"""星&nbsp;&nbsp;&nbsp;&nbsp;座</dt>
<dd class="basicInfo-item value">
<a target=["]?_blank["]? href="[\d\D]*?">([\d\D]*?)</a>
</dd>""")
            for match in reobj.finditer(html):
                sxz = match.group(1)
        elif '<sup class=' in sxz:
            sxz = sxz[:3]
        s.xz = sxz.replace("&nbsp;","")#去掉获取到的星座中的"&nbsp;"
        db.commit()

# 采集明星的身高并入库

In [ ]:
@db_session
def get_height(html):
    sheight = 0
    reobj = re.compile(r"""身&nbsp;&nbsp;&nbsp;&nbsp;高</dt>
<dd class="basicInfo-item value">
[&nbsp;]*?([\d.]*?)[cmCM厘米<sup ]{1,2}[\d\D]*?
</dd>""", re.DOTALL)
    for match in reobj.finditer(html):
        if len(match.group(1)) > 5:
            reobj = re.compile(r"""身&nbsp;&nbsp;&nbsp;&nbsp;高</dt>
<dd class="basicInfo-item value">
([\d\D]*?)<a target=["]?_blank["]? href=""")
            for match in reobj.finditer(html):
                sheight = match.group(1)
        elif match.group(1) == "":
            reobj = re.compile(r"""身&nbsp;&nbsp;&nbsp;&nbsp;高</dt>
<dd class="basicInfo-item value">
[&nbsp;]*?([\d.]*?)[cmCM厘米<sup ]{1,2}
</dd>""", re.DOTALL)
            for match in reobj.finditer(html):
                if str(match.group(1)).isdigit() == True:
                    sheight = match.group(1)
        else:
            sheight = match.group(1)
    if sheight != 0 :
        s.height = str(sheight).replace("&nbsp;","")#去掉获取到的身高中的"&nbsp;"
        db.commit()

# 采集明星的出生地并入库

In [ ]:
@db_session
def get_address(html):
    saddress = ""
    reobj = re.compile(r"""出生地</dt>
<dd class="basicInfo-item value">
([\d\D]*?)
</dd>""")
    for match in reobj.finditer(html):
        if match.group(1).count('<a target=') == 1:
            reobj = re.compile(r"""出生地</dt>
<dd class="basicInfo-item value">
([\d\D]*?)<a target=["]?_blank["]? href="[\d\D]*?">([\d\D]*?)</a>
</dd>""")
            for match in reobj.finditer(html):
                if len(match.group(1)) + len(match.group(2)) < 20:
                    saddress = match.group(1) + match.group(2)
                    
            reobj = re.compile(r"""出生地</dt>
<dd class="basicInfo-item value">
<a target=["]?_blank["]? href="[\d\D]*?">([\d\D]*?)</a>([\d\D]*?)
</dd>""")
            for match in reobj.finditer(html):
                if len(match.group(1)) + len(match.group(2)) < 20:
                    saddress = match.group(1) + match.group(2)
                    
                    
        elif match.group(1).count('<a target=') == 2:
            reobj = re.compile(r"""出生地</dt>
<dd class="basicInfo-item value">
<a target=["]?_blank["]? href="[\d\D]*?">([\d\D]*?)</a>[&nbsp;]*?([\d\D]*?)<a target=["]?_blank["]? href="[\d\D]*?">([\d\D]*?)</a>([\d\D]*?)
</dd>""")
            for match in reobj.finditer(html):
                if len(match.group(1)) + len(match.group(2))+ len(match.group(3)) + len(match.group(4)) < 20:
                    saddress = match.group(1) + match.group(2) + match.group(3) + match.group(4)
                
                
        elif match.group(1).count('<a target=') == 3:
            reobj = re.compile(r"""<a target=["]?_blank["]? href="/item/%E4%BA%91%E5%8D%97">([\d\D]*?)</a><a target=["]?_blank["]? href="[\d\D]*?">([\d\D]*?)</a><a target=["]?_blank["]? href="/item/%E6%A5%9A%E9%9B%84%E5%B8%82">([\d\D]*?)</a>
</dd>""")
            for match in reobj.finditer(html):
                if len(match.group(1)) + len(match.group(2)) +len(match.group(3)) < 20:
                    saddress = match.group(1) + match.group(2) + match.group(3)
                
                
        elif """<sup class="sup--normal" data-sup="1" data-ctrmap=":1,">""" in match.group(1):
            reobj = re.compile(r"""出生地</dt>
<dd class="basicInfo-item value">
([\d\D]*?)<sup class="sup--normal" data-sup="1" data-ctrmap=":1,">
\[1\]</sup><a class="sup-anchor" name="ref_\[1\]_10531">&nbsp;</a>
</dd>""")
            for match in reobj.finditer(html):
                if len(match.group(1))< 20:
                    saddress = match.group(1)
                
                
        elif """<dt class="basicInfo-item name">出生地</dt>""" in match.group(1):
            reobj = re.compile(r"""<dt class="basicInfo-item name">出生地</dt>
<dd class="basicInfo-item value">
<a target="[\d\D]*?">([\d\D]*?)</a>([\d\D]*?)<a target=["]?_blank["]? href="[\d\D]*?">([\d\D]*?)</a>([\d\D]*?)
</dd>""")
            for match in reobj.finditer(html):
                if len(match.group(1)) + len(match.group(2)) + len(match.group(3)) + len(match.group(4)) < 20:
                    saddress = match.group(1) + match.group(2) + match.group(3) + match.group(4)
                
                
        elif """<sup class=""" in match.group(1):
            reobj = re.compile(r"""出生地</dt>
<dd class="basicInfo-item value">
([\d\D]*?)<sup class="[\d\D]*?</a>
</dd>""")
            for match in reobj.finditer(html):
                if len(match.group(1)) < 20:
                    saddress = match.group(1)

                
        else:
            saddress = match.group(1)
    s.address = saddress.replace("&nbsp;","")#去掉获取到的出生地中的"&nbsp;"
    db.commit()

# 采集明星的出生年并入库

In [ ]:
@db_session
def get_year(html):
    syear = 0
    x = ""
    reobj = re.compile(r"""出生日期</dt>
<dd class="basicInfo-item value[\d\D]*?">
([\d\D]*?)[年:\-. /][\d\D]*?
</dd>""", re.DOTALL)
    for match in reobj.finditer(html):
        if match.group(1) is None:
            break
        elif "月" in match.group(1) or "日" in match.group(1):
            syear = 0
        elif len(match.group(1)) == 4:
            syear = match.group(1)
        elif "<a target=" in match.group(1):
            if str(match.group(1)[-4:]).isdigit() == True:
                syear = match.group(1)[-4:]
            else:
                reobj = re.compile(r"""出生日期</dt>
<dd class="basicInfo-item value">
<a target=[\d\D]*?">([\d\D]*?)</a>[年.\- /][\d\D]*?
</dd>""", re.DOTALL)
                for match in reobj.finditer(html):
                    if str(match.group(1)).isdigit() == True:
                        syear = match.group(1)
                    
    reobj = re.compile(r"""出生日期</dt>
<dd class="basicInfo-item value">
生于([\d\D]*?)[年:\-. /][\d\D]*?
</dd>""", re.DOTALL)
    for match in reobj.finditer(html):
        for i in match.group(1):
            if i in sz:
                x += str(sz.find(i)+1)
        syear = x
    if str(syear).isdigit() == True and syear != 0:
        s.year = str(syear).replace("&nbsp;","")#去掉获取到的出生年中的"&nbsp;"
        db.commit()

# 采集明星的出生月并入库

In [ ]:
@db_session
def get_month(html):
    smonth = 0
    reobj = re.compile(r"""出生日期</dt>
<dd class="basicInfo-item value">
[\d\D]*?[年:\-. /]?([\d\D]*?)[月\-./][\d\D]*?
</dd>""", re.DOTALL)
    for match in reobj.finditer(html):
        smonth = match.group(1)
        if '年' in match.group(1) and len(re.split('年',match.group(1))[1].strip()) < 6:
            if re.split('年',match.group(1))[1].strip().isdigit() == True:
                smonth = re.split('年',match.group(1))[1].strip()
            elif re.split('年',match.group(1))[1].strip() in sz:
                smonth = str(sz.find(re.split('年',match.group(1))[1])+1)
            elif '</a>' in match.group(1) or ""and len(match.group(1)) < 60:
                smonth = str(re.split('年',match.group(1))[1]).strip().strip('</a>')
            elif match.group(1).count("年") == 2:
                smonth = str(re.split('年',match.group(1))[2]).strip()[0:]
                if len(smonth) >2:
                    smonth = smonth[-2:]
            else:
                smonth = 0
        elif "历" in match.group(1) and len(str(re.split('历',match.group(1))[1]).strip()) < 3:
            if re.split('历',match.group(1))[1].strip().isdigit() == True:
                smonth = re.split('历',match.group(1))[1].strip()
            elif re.split('历',match.group(1))[1].strip() in sz:
                smonth = sz.find(str(re.split('历',match.group(1))[1]).strip())+1
        elif '<a target=' in smonth and len(str(smonth)) < 55:
            reobj = re.compile(r"""出生日期</dt>
<dd class="basicInfo-item value">
<a target=[\d\D]*?年([\d\D]*?)月[\d\D]*?""", re.DOTALL)
            for match in reobj.finditer(html):
                smonth = match.group(1)
        elif '<a target=' in smonth and len(str(smonth)) > 55:
            reobj = re.compile(r"""出生日期</dt>
<dd class="basicInfo-item value">
([\d\D]*?)月[\d\D]*?
</dd>""", re.DOTALL)
            for match in reobj.finditer(html):
                if str(match.group(1)[-2:]).isdigit() == True:
                    smonth = match.group(1)[-2:]
                else:
                    smonth = match.group(1)[-1:]
        elif str(smonth).isdigit == True and len(str(smonth)) == 4:
            reobj = re.compile(r"""出生日期</dt>
<dd class="basicInfo-item value">
[\d\D]*?[年:\-. /]([\d\D]*?)[月\-. /][\d\D]*?
</dd>""", re.DOTALL)
            for match in reobj.finditer(html):
                if str(match.group(1)).sidigit == True:
                    smonth = match.group(1)

        if len(match.group(1)) == 5 and '</a>' in match.group(1)and len(re.findall('[0-9]',match.group(1))[0])< 3:
            smonth = re.findall('[0-9]',match.group(1))[0]
        if str(smonth).isdigit() != True:
            smonth = 0
    if smonth != 0:
        s.month = str(smonth).replace("&nbsp;","")#去掉获取到的出生月中的"&nbsp;"
        db.commit()

# 采集明星的出生日并入库

In [ ]:
@db_session
def get_day(html):
    sday = 0
    x = ""
    reobj = re.compile(r"""出生日期</dt>
<dd class="basicInfo-item value">
[\d\D]*?[年:\-. /]?[\d\D]*?[月:\-. /]([\d\D]*?)[日.-号][&nbsp;]?[\d\D]*?
</dd>""", re.DOTALL)
    for match in reobj.finditer(html): 
        if match.group(1) is None or len(match.group(1))>2 or match.group(1) == "":
            sday = 0
        else:
            sday = match.group(1)
    
    reobj = re.compile(r"""出生日期</dt>
<dd class="basicInfo-item value">
[\d\D]*?[年-]?[\d\D]*?[月.-]([\d\D]{2}?)[日.-号][&nbsp;]?[\d\D]*?
</dd>""", re.DOTALL)
    for match in reobj.finditer(html):
        for i in match.group(1):
            if i.isdigit() == True:
                x += i
        if len(x) == 2:
            sday = match.group(1)
        else:
            reobj = re.compile(r"""出生日期</dt>
<dd class="basicInfo-item value">
[\d\D]*?[年-]?[\d\D]*?[月.-]([\d\D]{1}?)[日.-号][&nbsp;]?[\d\D]*?
</dd>""", re.DOTALL)
            for match in reobj.finditer(html):
                if match.group(1).strip().isdigit() == True:
                    sday = match.group(1).strip()
                else:
                    reobj = re.compile(r"""出生日期</dt>
<dd class="basicInfo-item value">
[\d\D]*?月([\d\D]*?)[日号]
</dd>""", re.DOTALL)
                    for match in reobj.finditer(html):
                        if match.group(1).isdigit() == True:
                            sday = match.group(1)
                        else:
                            reobj = re.compile(r"""出生日期</dt>
<dd class="basicInfo-item value">
[\d\D]*?[月:\-. /][\d\D]*?[月:\-. /]([\d\D]*?)
</dd>""", re.DOTALL)
                            for match in reobj.finditer(html):
                                if str(match.group(1)).isdigit == True:
                                    sday = match.group(1)
    if str(sday).isdigit() != True:
        sday = 0
    if sday != 0:
        s.day = str(sday).replace("&nbsp;","")#去掉获取到的出生日中的"&nbsp;"
        db.commit()

# 执行

In [ ]:
sz = "一二三四五六七八九"
with db_session:
    for s in Star.select():
        get_height(s.html)
        get_xz(s.html)
        get_address(s.html)
        get_year(s.html)
        get_month(s.html)
        get_day(s.html)